In [1]:
from experiments import BTI_DBF_U
import os
from functools import partial
from datasets import SimpleMNISTDataset, prepare_mnist_data, minmax_normalize
import torch
from unet import UNet, loss_bti_dbf_paper
from torchvision import transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Prepare data if not already present
print("Preparing MNIST dataset...")
prepare_mnist_data()

# Transforms (tensor only for the DataLoader; the BTI code handles its own normalize)
transform_test = transforms.ToTensor()

# U-Net config for MNIST (1 channel)
unet_factory = partial(UNet, n_channels=1, num_classes=1, base_filter_num=32, num_blocks=2)

# Create test dataset / loader
test_dataset = SimpleMNISTDataset(
    path_to_data='./MNIST_Data/clean',
    csv_filename='clean.csv',
    data_transform=transform_test
)
dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True, num_workers=2)
print(f"Test dataset size: {len(test_dataset)}")

# Variants
variants = [
    {
        "name": "paper-branch-1back",
        "train_fn": "branch",
        "unet_loss": loss_bti_dbf_paper,
        "tau": 3,
        "epochs": 30,
        "lr": 0.01,
        "p": 2,
        "visualize": True,
        "split": "1-back",
        "mask_granularity": "channel",
        "delta": True
    },
    {
        "name": "paper-branch-final",
        "train_fn": "branch",
        "unet_loss": loss_bti_dbf_paper,
        "tau": 3,
        "epochs": 30,
        "lr": 0.01,
        "p": 2,
        "visualize": True,
        "split": "final",
        "mask_granularity": "vector",
        "delta": True
    },
]

# Run BTI-DBF (U) with evaluation.
# Triggered datasets live under the usual root:
#   ./test_results/datasets/Odysseus-MNIST/Models/{model_file}_MNIST
BTI_DBF_U(
    device=device,
    num_models=5,
    model_list='./test_results/comprehensive_test_results_20250813_173218.csv',
    model_dir='./Odysseus-MNIST/Models',
    model_type='MNIST',
    unet_factory=unet_factory,
    dataloader=dataloader,
    variants=variants,
    mask_epochs=20,
    # evaluation paths
    triggered_dataset_root="./test_results/datasets",
    # unlearning params (explicit for clarity; you can omit to use defaults)
    unlearn_epochs=3,
    unlearn_lr=1e-3,
    feature_loss_weight=1.0,
    alt_rounds=1,
)

Using device: cuda
Preparing MNIST dataset...


clean: 100%|█████████████████████████████| 8000/8000 [00:00<00:00, 14313.00it/s]


Saved 8000 images to ./MNIST_Data/clean
Saved CSV to ./MNIST_Data/clean/clean.csv


test: 100%|██████████████████████████████| 2000/2000 [00:00<00:00, 14317.08it/s]


Saved 2000 images to ./MNIST_Data/test
Saved CSV to ./MNIST_Data/test/test.csv
Test dataset size: 8000


...

[U] round 0 mask raw init = tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.

/home/tyler/Desktop/ResearchProject/Load_Model.py:322: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(output)


Evaluating model: ./Odysseus-MNIST/Models/Model_901.pth
Dataset directory: ./test_results/datasets/Odysseus-MNIST/Models/Model_901.pth_MNIST
Dataset statistics:
  Total images: 2000
  Triggered images: 1000
  Clean images: 1000


/home/tyler/Desktop/ResearchProject/Load_Model.py:322: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(output)
BTI-DBF (U): Models: 100%|████████████████████████| 5/5 [06:21<00:00, 76.26s/it]

✅ BTI-DBF (U) results saved to MNIST_BTI_DBF_U_results.csv
